In [1]:
# BASIC IMPORTS OF ALL LIBRARIES
'''
Glove Imports are 300-dim from GENSIM
Pickle saves all the datasets
'''
from nltk import pos_tag
import csv
import re
import numpy as np
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from collections import defaultdict
import networkx as nx
from copy import deepcopy
import pickle
from entity_groups import *
from graphviz import Source
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist 
import gensim.downloader as api

wnl = WordNetLemmatizer()
word_vectors = api.load("glove-wiki-gigaword-300")

In [38]:
# junk verbs that are generalizably illegal
junk_rel = ["be","have","do","let","","make","tell","say","describe","decide","want","name","know","think","try","become","oneday","put","come",'see', 'need', 'look', 'help', 'come', 'take', 'get', 'put', 'pick', 'turn', 'go', 'stand', 'give', 'notice', 'use',"get","start"]

# hypothetical words that are generally useless / qualifiers
junk_words = ["would","could","should","maybe","perhaps","think","might","assume","claim","this"]

# negative sentences
junk_words.extend(['n\'t',"not"])
print(junk_words)

# perspective actants that are pointless
junk = ["you","i","we","the","it","he","she","steinbeck","people","author",
        "book","me","steinback","him","her","their","this","","shelley","mary","harper","lee","tolkien"]

['would', 'could', 'should', 'maybe', 'perhaps', 'think', 'might', 'assume', 'claim', 'this', "n't", 'not']


In [39]:
########################## EDIT DATE ##########################

DATE = "0413"
file_name = "OS_News/"
text = "OS_News"

In [40]:
import pandas as pd
results_path = "Results" + DATE + "/" + text
adj_mat_path = "Precedence_Matrix/" + text
abs_adj_mat_path = "AbsPrecedence_Matrix/" + text
dic_path = "Dictionary/" + text + ".pickle"
ner_file = "../Covid_19_Data/Extracted_Data/OS_News/" + "into_relex_final_relations_-1.csv"
entity_file = "../Covid_19_Data/Extracted_Data/OS_News/" + "df_ent_final_ranking.csv"
S_use = {}
# fill this in
ent_file = pd.read_csv(entity_file)
for row in ent_file.head(n=20).iterrows():
    S_use[row[1]['entity']] = row[1]['entity']

fileName = ner_file

In [41]:
# HELPER FUNCTION
def hypothetical_sentence(sent):
    '''
    Removes sentences that have the hypothetical list of words in them. 
    Does not offer information about sequencing.
    '''
    for word in sent.split():
        for ground_word in junk_words:
            if ground_word in word:
                return True
    return False

In [42]:
trajectories = []
same_post = 0
traj = ["START"]

with open(fileName) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')

    for row in readCSV:
        FLAG = 0
        sentence_num = row[1]
        sub_r = ""
        obj_r = ""
        sub_verb = ""

        if hypothetical_sentence(row[2]):
            continue

        try:
            sub = re.search(r"\{(\w+)\}", row[3]).group(1).lower()
            rel = wnl.lemmatize(re.search(r"\{(\w+)\}", row[4]).group(1).lower(), pos='v')
            obj = re.search(r"\{(\w+)\}", row[5]).group(1).lower()
        except:
            continue

        words = row[2].split()
        rel_words = []

        if rel == None or obj == None or sub == None:
            continue
        
        if same_post == row[0]:
            pass
        else:
            same_post = row[0]
            traj.append("TERMINATE")
            trajectories.append(traj)
            traj = ["START"]

        try:
            sub_r = S_use[sub]
        except:
            sub_r = sub
            FLAG += 1
        
        try:
            obj_r = S_use[obj]
        except:
            obj_r = obj
            FLAG += 1

        sub_verb = ""

        try:
            sub_verb = C_use[rel]
        except:
            sub_verb = rel

        if sub_r == obj_r or sub_r in junk or obj_r in junk or sub_verb in junk_rel \
                                or FLAG == 2 or sub_r.isdigit() or obj_r.isdigit():
            continue

        traj.append(sub_verb)
        traj.append(sub_r + "_" + obj_r + "_" + sentence_num)

# if we missed out on last post
if traj != "START":
    traj.append("TERMINATE")
    trajectories.append(traj)

print(trajectories)

e', 'us_shortages_100', 'struggle', 'us_shortages_100', 'TERMINATE'], ['START', 'include', 'death_cases_5', 'TERMINATE'], ['START', 'view', 'trump_coronavirus_1', 'view', 'trump_serious_1', 'call', 'coronavirus_pandemic_1', 'view', 'trump_coronavirus_1', 'write', 'trump_year_3', 'read', 'trump_something_9', 'read', 'trump_something_9', 'downplay', 'trump_impact_11', 'provide', 'trump_hope_11', 'provide', 'coronavirus_hope_11', 'extend', 'trump_centers_12', 'extend', 'trump_control_12', 'extend', 'trump_centers_12', 'TERMINATE'], ['START', 'test', 'third_virus_24', 'test', 'third_virus_25', 'apologise', 'chief_italy_32', 'apologise', 'chief_italy_32', 'mobilise', 'europe_italy_33', 'die', 'large_coronavirus_36', 'contract', 'large_coronavirus_36', 'contract', 'workers_virus_43', 'die', 'workers_virus_43', 'contract', 'workers_italy_43', 'contract', 'workers_virus_43', 'deplete', 'stocks_us_46', 'admit', 'trump_stocks_46', 'TERMINATE'], ['START', 'impose', 'measures_virus_12', 'fight', '

In [43]:
precedence_matrix = np.zeros((50000,50000))
dictionary_of_labels = {"START":0, "TERMINATE":1}
counter_of_labels = defaultdict(int)
max_rows = 1
max_cols = 1
counter = 2

for trajectory in trajectories:

    triplets = ["START"]
    index = 1

    while index < len(trajectory)-2:
        triplets.append(trajectory[index] + '_' + trajectory[index + 1])
        index += 2

    triplets.append("TERMINATE")

    for key,_ in enumerate(triplets[1:-1]):
        removed_triplet = triplets[key+1].rsplit("_",1)[0]

        if removed_triplet not in dictionary_of_labels:
            dictionary_of_labels[removed_triplet] = counter
            counter += 1
    
    triplet_temp = []

    # find uniques
    for trip in triplets:
        try:
            removed_triplet = trip.rsplit("_",1)[0]
        except:
            removed_triplet = trip

        if removed_triplet not in [t.rsplit("_",1)[0] for t in triplet_temp]:
            triplet_temp.append(trip)

    triplets = triplet_temp

    # for trip in triplets:
    #     counter_of_labels[trip] += 1

    alpha = 1.0

    for key in range(1,len(triplets)):
        key_hist = key-1

        while key_hist >= 0:
            removed_triplet_hist = triplets[key_hist].rsplit("_",1)
            removed_triplet_now = triplets[key].rsplit("_",1)
            try:
                max_cols = max(max_cols, dictionary_of_labels[removed_triplet_now[0]])
            except:
                pass

            try:
                if removed_triplet_hist[1] != removed_triplet_now[1]:
                    precedence_matrix[dictionary_of_labels[removed_triplet_hist[0]]]\
                                        [dictionary_of_labels[removed_triplet_now[0]]] += alpha
            except:
                precedence_matrix[dictionary_of_labels[removed_triplet_hist[0]]]\
                                        [dictionary_of_labels[removed_triplet_now[0]]] += alpha

            alpha /= 1.0
            key_hist -= 1

precedence_matrix = precedence_matrix[:max_cols+1,:max_cols+1]

In [44]:
# Relationship Clustering - Helper Function
mean_threshold = 3.0

def ordered_elbowkmeans_with_glove_and_labels(verbs, weights):
    output_temp = []
    vecs = []
    distortions = []
    degree = 1

    for i,verb in enumerate(verbs):
        for j in range(weights[i]):
            output_temp.append(verb)
            try:
                vecs.append(list(word_vectors[verb]))
            except:
                vecs.append([-1000]*300) # junk vector that won't affect the KMean

    vecs = np.array(vecs)

    for k in range(100): 
        try:
            kmeanModel = KMeans(n_clusters=(k+1)).fit(vecs) 
            if sum(np.min(cdist(vecs, kmeanModel.cluster_centers_, 
                        'euclidean'),axis=1)) / vecs.shape[0] < mean_threshold:
                        degree = k+1
                        break
        except:
            print("Elbow did not work!")
            break
    try:
        kmeanModel.predict(vecs)
        labels = kmeanModel.labels_

        for label in list(set(labels)):

            positions = np.argwhere(labels == label)
            min_dist = 10000000
            index = -1
            dist = 0

            for k,position in enumerate(positions):
                sample = np.min(cdist(vecs[position[0]].reshape(1,-1), kmeanModel.cluster_centers_, 'euclidean'),axis=1)
                dist += sample
                if min_dist > sample:
                    min_dist = sample
                    index = k

            if dist / len(positions) > mean_threshold*2:
                continue

            if list(vecs[positions[0][0]]) == [-1000]*300: # resolved here
                continue

            for peroni in positions:
                output_temp[peroni[0]] = output_temp[positions[index][0]] # closest to the center
    except:
        output_temp = output_temp

    output = []
    for val in output_temp:
        if val not in output:
            output.append(val)

    return output

In [45]:
new_mat = deepcopy(precedence_matrix)
new_dict = defaultdict(list)
rel_map = defaultdict(list)
other_dict = dictionary_of_labels
diction = {}

for key in other_dict:
    diction[other_dict[key]] = key

for key in diction:
    if diction[key] not in ["START","TERMINATE"]:
        splits = diction[key].partition("_")
        new_dict[splits[2]].append(splits[0])

for key in new_dict:
    verbs = new_dict[key]
    weights = []

    for verb in verbs:
        weights.append(counter_of_labels[verb + "_" + key])
    labels = ordered_elbowkmeans_with_glove_and_labels(verbs, weights)

    for num in range(len(labels)):
        rel_map[other_dict[labels[num] + "_" + key]].append(other_dict[verbs[num] + "_" + key])

#print([diction[i].partition("_")[0] for i in rel_map[other_dict["come_george_lennie"]]])

counter_of_labels_t = defaultdict(int)
for key in rel_map:

    for val in rel_map[key]:
        counter_of_labels_t[diction[key]] += counter_of_labels[diction[val]]
        if key != val:
            new_mat[key] = new_mat[key] + new_mat[val]
            new_mat[val] = 0

    new_mat_t = np.transpose(new_mat)

    for val in rel_map[key]:
        if key != val:
            new_mat_t[key] = new_mat_t[key] + new_mat_t[val]
            new_mat_t[val] = 0
            
    new_mat = np.transpose(new_mat_t)

Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!
Elbow did not work!


In [46]:
for i in range(new_mat.shape[0]):
    new_mat[i][i] = 0.0

In [47]:
val_precedence_matrix = deepcopy(new_mat)

for row in new_mat:
    row[1] = 1.0/1000000.0

for i in range(len(new_mat[0])):
    if i >= 2:
        new_mat[0][i] = 1.0 / len(new_mat[0]-2)

for row in new_mat:
    sumt = sum(row)
    row /= sumt

In [48]:
mat = deepcopy(new_mat)
val_mat = deepcopy(val_precedence_matrix)

mat_s = deepcopy(mat)
mat_s[mat_s > 0.0] = 1
mat_st = np.transpose(mat_s)

singles = pair_wise_cycles = 0

# 1. remove pair-wise cycles
for i in range(0, mat.shape[0]):
    for j in range(0, mat.shape[1]):

        if mat[i][j] > mat[j][i]:
            if mat[j][i] != 0:
                pair_wise_cycles += 1.0
            mat[j][i] = 0

        elif mat[i][j] < mat[j][i]:
            if mat[i][j] != 0:
                pair_wise_cycles += 1.0
            mat[i][j] = 0

        elif mat[i][j] == mat[j][i] and i != j and mat[j][i] != 0:
            mat[j][i] = 0
            pair_wise_cycles += 1.0
            mat[i][j] = 0

#print("Number of Pair-Wise Cycles: ", pair_wise_cycles)
#print("Richness: ", np.sum(mat))

for i in range(0, mat.shape[1]):
    # no branches from triplet for TERMINATE AND START
    if np.sum(mat_st[i]) == 1 and np.sum(mat_s[i] == 1): 
        mat[i] = 0.0
        mat[0][i] = 0.0
        singles += 1.0

# We do not want to go from START -> TERMINATE directly.
mat[0][1] = 0.0
mat[1][1] = 0.0

#print("Number of Single Triplets: ", singles)

In [49]:
def compute_metric(from_node, to_node, mat):
    '''
    Adjust the metric to keep in check the approach of the graph.
    '''
    trust = mat[from_node][to_node]
    mat_t = np.transpose(mat)
    in_degree = np.sum(mat_t[to_node] > 0)
    out_degree = np.sum(mat[to_node] > 0)
    #print(trust, out_degree + in_degree, val_mat[from_node][to_node])
    return trust + 0.001*(out_degree + in_degree) + 0.01*val_mat[from_node][to_node]
    #return trust*(out_degree + in_degree)


def not_cycle_from_to_node_to_from_node(from_node, to_node, mat_flow):
    '''
    Simple BFS: we do know that there are no cycles in here so
    no worries in computing a path from to to, to detemine cycle
    '''
    visited =[False]*mat_flow.shape[0] 
    #Create a queue for BFS 
    queue=[] 
    # Mark the source node as visited and enqueue it 
    queue.append(from_node) 
    visited[from_node] = True
    while queue: 
        #Dequeue a vertex from queue  
        n = queue.pop(0)      
        # If this adjacent node is the destination node, 
        # then return true 
        if n == to_node: 
            return False
        #  Else, continue to do BFS 
        to_nodes = np.argwhere(mat_flow[n] != 0)
        for i in to_nodes: 
            i = i[0]
            if visited[i] == False: 
                queue.append(i) 
                visited[i] = True
    # If BFS is complete without visited d 
    return True

In [50]:
number_of_steps = 100
current_time = 0
stack_per_time = [0]
cycles = 0
cycles_prob = []
weight_of_discarded_edge = []

time_step = np.zeros_like(mat[0]) - 1.0
mat_flow = np.zeros_like(mat)

while current_time < number_of_steps:

    print(current_time)
    if stack_per_time == []:
        print("Max Time Limit: ", current_time)
        break
    
    new_stack = []
    for start_node in stack_per_time:
        metric = []
        to_nodes = np.argwhere(mat[start_node] != 0)

        for to_node in to_nodes:
            to_node = to_node[0]
            metric.append(compute_metric(start_node, to_node, mat))

        # what happens to those with no to-nodes like TERMINATE   
        try:
            n_metric = np.argsort(-1*np.array(metric))
            threshold = metric[n_metric[0]] * 0.05
        except:
            n_metrc = []
            threshold = 1000

        for i in range(len(n_metric)):
            if metric[n_metric[i]] > threshold:
                to_node_correct = to_nodes[n_metric[i]][0]

                if not_cycle_from_to_node_to_from_node(to_node_correct, start_node, mat_flow):
                    if to_node_correct not in new_stack:
                        new_stack.append(to_node_correct)
                    mat_flow[start_node][to_node_correct] = 1.0
                    time_step[to_node_correct] = current_time + 1.0
                    
                else:
                    if mat[start_node][to_node_correct] != 0:
                        cycles += 1
                        weight_of_discarded_edge.append(val_mat[start_node][to_node_correct])
                        # print(mat[start_node][to_node_correct], val_mat[start_node][to_node_correct], start_node, to_node_correct)

    stack_per_time = new_stack
    print(stack_per_time)
    current_time += 1

print("Total ignored cycles: ", cycles)
print("Weights of Removed Edges: ", weight_of_discarded_edge)

0
[86, 5706, 5306, 574, 2680, 2781, 4744, 2009, 507, 5684, 5851, 3825, 651, 1180, 836, 558, 993, 4009, 5695, 1067, 2695, 112, 1177, 623, 154, 3162, 1641, 3590, 3482, 85, 4476, 4313, 1657, 420, 4170, 7862, 667, 551, 4710, 1151, 126, 8006, 6659, 5441, 2275, 1736, 767, 1995, 2704, 6487, 7842, 5896, 1665, 3115, 3114, 471, 3111, 3104, 3100, 3103, 2713, 3107, 3109, 3108, 3290, 1146, 2696, 2402, 1368, 2298, 975, 7869, 6513, 6519, 8026, 6493, 1020, 2992, 6366, 1320, 5649, 6060, 7867, 18, 452, 3099, 6245, 1575, 27, 7516, 5138, 4807, 7493, 450, 25, 4388, 5955, 8446, 3442, 5733, 1308, 5676, 2705, 2693, 2707, 2533, 6275, 2706, 783, 820, 1982, 3410, 7070, 4030, 1747, 68, 4770, 4184, 1994, 6259, 4216, 9626, 3125, 294, 5690, 6254, 2518, 5438, 7976, 622, 5354, 3617, 6032, 3404, 978, 985, 4041, 4042, 1023, 5674, 2157, 960, 4040, 2392, 4039, 4031, 4032, 4033, 4045, 6084, 5632, 26, 2011, 7429, 5682, 4161, 5227, 3116, 3101, 3102, 3110, 6980, 5747, 3375, 6174, 6928, 1339, 324, 6534, 2266, 10037, 2262, 4543

In [51]:
for i in range(len(time_step)):
    for j in range(len(time_step)):
        if time_step[i] < time_step[j] and time_step[i] != -1 and time_step[j] != -1 and mat_flow[i][j] != 0:
            for k,source_node in enumerate(mat_flow[:][j]):
                if mat_flow[k][j] > 0 and k != i:
                    if not not_cycle_from_to_node_to_from_node(k,i, mat_flow):
                        mat_flow[k][j] = 0

In [52]:
for i in range(0, mat_flow.shape[1]):
    # no branches from triplet for TERMINATE AND START
    if mat_flow[0][i] != 0 and mat_flow[i][1] != 0: 
        mat_flow[0][i] = 0.0
        mat_flow[i][1] = 0.0

In [53]:
G = nx.from_numpy_matrix(mat_flow, create_using=nx.DiGraph)
DISP_THRESH = 2
bb = {}
for index, val in np.ndenumerate(val_mat):
    if val > 1:
        bb[index] = {"color":"red", "penwidth":2}
    else:
        bb[index] = {"color":"blue"}
nx.set_edge_attributes(G, bb)

bb1 = {}
for i in range(mat_flow.shape[0]):
    if np.sum(mat_flow[i]) + np.sum(np.transpose(mat_flow[i])) > DISP_THRESH:
        bb1[i] = {"color":"red", "penwidth":2}
    else:
        bb1[i] = {"color":"blue"}
nx.set_node_attributes(G, bb1)

In [57]:
import pygraphviz
H = nx.relabel_nodes(G, diction)
H.remove_nodes_from(list(nx.isolates(H)))
A = nx.nx_agraph.to_agraph(H)

for i in range(number_of_steps):
    same_priority = []
    for j in range(len(time_step)):
        if time_step[j] == i:
            same_priority.append(diction[j])
    one = A.add_subgraph(same_priority,rank='same')

A.layout('dot')
A.draw(results_path + '.png')